In [1]:
import numpy as np
import pandas as pd
# We will use the sklearn preprocessing library, as it will be easier to standardize the data.
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

# Load the data
raw_excel_data = pd.read_excel('Database.xlsx')

# Replace missing values in the columns with their respective mean
#raw_excel_data.fillna(raw_excel_data.mean(), inplace=True)

raw_excel_data.dropna(inplace=True)

In [2]:
# Check the number of missing values in each column
missing_values = raw_excel_data.isnull().sum()

# Display columns with missing values and their counts
missing_values[missing_values > 0]

Series([], dtype: int64)

In [3]:
unscaled_inputs_all = raw_excel_data.iloc[:,0:-1].values

# The targets are in the last column. That's how datasets are conventionally organized.
unscaled_targets_all = raw_excel_data.iloc[:,-1].values

In [5]:
# Define the StandardScaler object for inputs and targets
#scaler_inputs = preprocessing.StandardScaler()
#scaler_targets = preprocessing.StandardScaler()

# Initialize the scaler
scaler = MinMaxScaler(feature_range=(-1, 1))

# Continue with scaling and calculations
scaled_inputs = scaler.fit_transform(unscaled_inputs_all)
scaled_targets = scaler.fit_transform(unscaled_targets_all.reshape(-1, 1))
print(scaled_inputs[:5])
print(scaled_targets[:5])

[[-0.01190476 -1.         -1.         -1.          0.39534884  0.42779292
  -1.         -1.         -0.11164466 -1.         -1.        ]
 [-0.01190476 -1.         -1.         -1.          0.39534884  0.14441417
  -0.68125    -0.32347826 -0.11164466 -1.         -1.        ]
 [-0.01190476 -1.         -1.         -1.          0.39534884 -0.28610354
  -0.209375   -0.32347826 -0.11164466 -1.         -1.        ]
 [-0.01190476 -1.         -1.         -1.          0.39534884 -1.
   0.5890625  -0.32347826 -0.11164466 -1.         -1.        ]
 [-0.31696429 -1.         -1.         -0.45333333  0.39534884  0.42779292
  -1.         -1.         -0.18607443 -1.         -1.        ]]
[[-0.87267736]
 [-0.88700115]
 [-0.89177575]
 [-0.91405722]
 [-0.88381809]]


In [6]:
# When the data was collected it was actually arranged by date
# Shuffle the indices of the data, so the data is not arranged in any way when we feed it.
# Since we will be batching, we want the data to be as randomly spread out as possible
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the inputs and targets.
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = scaled_targets [shuffled_indices]

In [7]:
import tensorflow as tf
# Count the total number of samples
samples_count = shuffled_inputs.shape[0]

# Count the samples in each subset, assuming we want 80-10-10 distribution of training, validation, and test.
# Naturally, the numbers are integers.
train_samples_count = int(0.7 * samples_count)
validation_samples_count = int(0.15 * samples_count)

# The 'test' dataset contains all remaining data.
test_samples_count = samples_count - train_samples_count - validation_samples_count

# Create variables that record the inputs and targets for training
# In our shuffled dataset, they are the first "train_samples_count" observations
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# Create variables that record the inputs and targets for validation.
# They are the next "validation_samples_count" observations, folllowing the "train_samples_count" we already assigned
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

# Create variables that record the inputs and targets for test.
# They are everything that is remaining.
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

# We balanced our dataset to be 50-50 (for targets 0 and 1), but the training, validation, and test were 
# taken from a shuffled dataset. Check if they are balanced, too. Note that each time you rerun this code, 
# you will get different values, as each time they are shuffled randomly.
# Normally you preprocess ONCE, so you need not rerun this code once it is done.
# If you rerun this whole sheet, the npzs will be overwritten with your newly preprocessed data.


Couldn't import dot_parser, loading of dot files will not be possible.


In [8]:
input_size = 12
output_size = 1
hidden_layer_size = 200

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size, activation ='sigmoid'),
                            tf.keras.layers.Dense(hidden_layer_size, activation ='sigmoid'),
                            tf.keras.layers.Dense(output_size, activation ='linear'),
                            ])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss ='mean_squared_error', metrics =['mse'])

batch_size = 128
max_epochs = 1000
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 20)
model.fit (train_inputs,
          train_targets,
          batch_size = batch_size,
          epochs = max_epochs,
          callbacks=[early_stopping],
          validation_data = (validation_inputs, validation_targets),
          verbose = 2)

Epoch 1/1000
17/17 - 1s - loss: 0.2140 - mse: 0.2140 - val_loss: 0.1227 - val_mse: 0.1227 - 599ms/epoch - 35ms/step
Epoch 2/1000
17/17 - 0s - loss: 0.1019 - mse: 0.1019 - val_loss: 0.0955 - val_mse: 0.0955 - 60ms/epoch - 4ms/step
Epoch 3/1000
17/17 - 0s - loss: 0.0863 - mse: 0.0863 - val_loss: 0.0796 - val_mse: 0.0796 - 51ms/epoch - 3ms/step
Epoch 4/1000
17/17 - 0s - loss: 0.0787 - mse: 0.0787 - val_loss: 0.0767 - val_mse: 0.0767 - 50ms/epoch - 3ms/step
Epoch 5/1000
17/17 - 0s - loss: 0.0777 - mse: 0.0777 - val_loss: 0.0753 - val_mse: 0.0753 - 51ms/epoch - 3ms/step
Epoch 6/1000
17/17 - 0s - loss: 0.0782 - mse: 0.0782 - val_loss: 0.0736 - val_mse: 0.0736 - 51ms/epoch - 3ms/step
Epoch 7/1000
17/17 - 0s - loss: 0.0743 - mse: 0.0743 - val_loss: 0.0717 - val_mse: 0.0717 - 54ms/epoch - 3ms/step
Epoch 8/1000
17/17 - 0s - loss: 0.0725 - mse: 0.0725 - val_loss: 0.0700 - val_mse: 0.0700 - 53ms/epoch - 3ms/step
Epoch 9/1000
17/17 - 0s - loss: 0.0719 - mse: 0.0719 - val_loss: 0.0689 - val_mse: 0.0

Epoch 73/1000
17/17 - 0s - loss: 0.0565 - mse: 0.0565 - val_loss: 0.0565 - val_mse: 0.0565 - 50ms/epoch - 3ms/step
Epoch 74/1000
17/17 - 0s - loss: 0.0552 - mse: 0.0552 - val_loss: 0.0569 - val_mse: 0.0569 - 53ms/epoch - 3ms/step
Epoch 75/1000
17/17 - 0s - loss: 0.0569 - mse: 0.0569 - val_loss: 0.0562 - val_mse: 0.0562 - 56ms/epoch - 3ms/step
Epoch 76/1000
17/17 - 0s - loss: 0.0570 - mse: 0.0570 - val_loss: 0.0624 - val_mse: 0.0624 - 63ms/epoch - 4ms/step
Epoch 77/1000
17/17 - 0s - loss: 0.0589 - mse: 0.0589 - val_loss: 0.0570 - val_mse: 0.0570 - 58ms/epoch - 3ms/step
Epoch 78/1000
17/17 - 0s - loss: 0.0561 - mse: 0.0561 - val_loss: 0.0560 - val_mse: 0.0560 - 52ms/epoch - 3ms/step
Epoch 79/1000
17/17 - 0s - loss: 0.0572 - mse: 0.0572 - val_loss: 0.0650 - val_mse: 0.0650 - 70ms/epoch - 4ms/step
Epoch 80/1000
17/17 - 0s - loss: 0.0636 - mse: 0.0636 - val_loss: 0.0641 - val_mse: 0.0641 - 47ms/epoch - 3ms/step
Epoch 81/1000
17/17 - 0s - loss: 0.0577 - mse: 0.0577 - val_loss: 0.0586 - val_m

In [9]:
# Predict the test targets using the trained model
test_predictions = model.predict(test_inputs)
test_targets_reshaped= test_targets.reshape(-1, 1)

# Unscale the predictions and targets
test_predictions_unscaled = scaler.inverse_transform(test_predictions)
test_targets_unscaled = scaler.inverse_transform(test_targets.reshape(-1, 1))

# Calculate the MAPE
mape = np.mean(np.abs((test_targets_unscaled - test_predictions_unscaled) / test_targets_unscaled)) * 100

# Calculate the RMSE
mse = np.mean((test_targets_unscaled - test_predictions_unscaled)**2)

# Calculate the MAE
mae = np.mean(np.abs(test_targets_unscaled - test_predictions_unscaled))

# Calculate the correlation coefficient (R)
r = np.corrcoef(test_targets_unscaled.T, test_predictions_unscaled.T)[0,1]

# Print the evaluation metrics
print('MAPE:', mape)
print('MSE:', mse)
print('MAE:', mae)
print('R:', r)

15/15 [==============================] - 0s 1ms/step
MAPE: 37.36519902534323
MSE: 233.68769711139
MAE: 12.277608558599532
R: 0.6207177841029861


In [ ]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import tensorflow as tf

# Define the hyperparameter space
space = {
    'hidden_layers': hp.choice('hidden_layers', [1, 2, 3, 4]),  # for simplicity, up to 4 hidden layers
    'hidden_layer_size': hp.choice('hidden_layer_size', [50, 100, 200, 400]),
    'activation': hp.choice('activation', ['relu', 'sigmoid', 'tanh']),
    'learning_rate': hp.loguniform('learning_rate', -7, -3)
}

# Define the objective function
def objective(params):
    model = tf.keras.Sequential()

    # Add the hidden layers
    for _ in range(params['hidden_layers']):
        model.add(tf.keras.layers.Dense(params['hidden_layer_size'], activation=params['activation']))

    # Output layer
    model.add(tf.keras.layers.Dense(output_size, activation='linear'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mse'])

    # You can use the early stopping callback if you want
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=10)

    history = model.fit(train_inputs,
                        train_targets,
                        batch_size=128,  # keeping this fixed for now, can be added to space
                        epochs=1000,  # keeping this fixed for now
                        callbacks=[early_stopping],
                        validation_data=(validation_inputs, validation_targets),
                        verbose=0)  # no output

    # Return the validation error from the last epoch
    val_error = history.history['val_mse'][-1]
    return {'loss': val_error, 'status': STATUS_OK}

# Run the optimization
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,  # for example
            trials=trials)

print(best)

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import tensorflow as tf

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Continue with the Bayesian Optimization and ANN as provided...

# Define the hyperparameter space
space = {
    'hidden_layers': hp.choice('hidden_layers', [1, 2, 3, 4]),  # for simplicity, up to 4 hidden layers
    'hidden_layer_size': hp.choice('hidden_layer_size', [50, 100, 200, 400]),
    'activation': hp.choice('activation', ['relu', 'sigmoid', 'tanh']),
    'learning_rate': hp.loguniform('learning_rate', -7, -3),
    'batch_size': hp.choice('batch_size', [32, 64, 128, 256]),
    'learning_rate_schedule': hp.choice('learning_rate_schedule', ['constant', 'step_decay', 'exponential_decay'])
}

# Define the objective function
def objective(params):
    model = tf.keras.Sequential()

    # Add the hidden layers
    for _ in range(params['hidden_layers']):
        model.add(tf.keras.layers.Dense(params['hidden_layer_size'], activation=params['activation']))

    # Output layer
    model.add(tf.keras.layers.Dense(output_size, activation='linear'))
    
    # Adjust learning rate based on schedule
    if params['learning_rate_schedule'] == 'step_decay':
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            params['learning_rate'],
            decay_steps=1000,
            decay_rate=0.9,
            staircase=True)
    elif params['learning_rate_schedule'] == 'exponential_decay':
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            params['learning_rate'],
            decay_steps=1000,
            decay_rate=0.9,
            staircase=False)
    else:
        lr_schedule = params['learning_rate']

    optimizer = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mse'])

    # You can use the early stopping callback if you want
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=20)

    history = model.fit(train_inputs,
                        train_targets,
                        batch_size=params['batch_size'],  # keeping this fixed for now, can be added to space
                        epochs=1000,  # keeping this fixed for now
                        callbacks=[early_stopping],
                        validation_data=(validation_inputs, validation_targets),
                        verbose=0)  # no output

    val_error = history.history['val_mse'][-1]

    # Return the validation error from the last epoch
    val_error = history.history['val_mse'][-1]
    return {
        'loss': val_error, 
        'status': STATUS_OK, 
        'mse': history.history['mse'], 
        'val_mse': history.history['val_mse']
    }

# Run the optimization
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,  # for example
            trials=trials)

# Extract mse and val_mse from the best trial
best_trial_index = np.argmin(trials.losses())
mse_best_trial = trials.trials[best_trial_index]['result']['mse']
val_mse_best_trial = trials.trials[best_trial_index]['result']['val_mse']

# Convert to dataframe
df_mse_best = pd.DataFrame({'mse': mse_best_trial})
df_val_mse_best = pd.DataFrame({'val_mse': val_mse_best_trial})

# Store data to Excel
with pd.ExcelWriter('optimisation_store.xlsx') as writer:
    df_mse_best.to_excel(writer, sheet_name='MSE', index=False)
    df_val_mse_best.to_excel(writer, sheet_name='Validation MSE', index=False)

print(best)

# After Bayesian Optimization, retrain the model with best hyperparameters
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

best_params = {
    'hidden_layers': [1, 2, 3, 4][best['hidden_layers']],
    'hidden_layer_size': [50, 100, 200, 400][best['hidden_layer_size']],
    'activation': ['relu', 'sigmoid', 'tanh'][best['activation']],
    'learning_rate': best['learning_rate'],
    'batch_size': [32, 64, 128, 256][best['batch_size']],
    'learning_rate_schedule': ['constant', 'step_decay', 'exponential_decay'][best['learning_rate_schedule']]
}

best_model = tf.keras.Sequential()

# Add the hidden layers
for _ in range(best_params['hidden_layers']):
    best_model.add(tf.keras.layers.Dense(best_params['hidden_layer_size'], activation=best_params['activation']))

# Output layer
best_model.add(tf.keras.layers.Dense(output_size, activation='linear'))

# Adjust learning rate based on best schedule
if best_params['learning_rate_schedule'] == 'step_decay':
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        best_params['learning_rate'],
        decay_steps=1000,
        decay_rate=0.9,
        staircase=True)
elif best_params['learning_rate_schedule'] == 'exponential_decay':
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        best_params['learning_rate'],
        decay_steps=1000,
        decay_rate=0.9,
        staircase=False)
else:
    lr_schedule = best_params['learning_rate']
    
optimizer = tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate'])
best_model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mse'])

early_stopping = tf.keras.callbacks.EarlyStopping(patience=20)

best_model.fit(train_inputs,
               train_targets,
               batch_size=best_params['batch_size'],
               epochs=1000,
               callbacks=[early_stopping],
               validation_data=(validation_inputs, validation_targets),
               verbose=2)

# Make predictions on test_inputs
test_predictions = best_model.predict(test_inputs)

# Calculate MAPE
mape = np.mean(np.abs((test_targets - test_predictions.flatten()) / test_targets)) * 100

# Calculate RMSE
mse = mean_squared_error(test_targets, test_predictions)

# Calculate MAE
mae = mean_absolute_error(test_targets, test_predictions)

# Calculate R^2
r = np.corrcoef(test_targets_reshaped.T, test_predictions.T)[0,1]

# Print the evaluation metrics
print('MAPE:', mape)
print('MSE:', mse)
print('MAE:', mae)
print('R:', r)